# quick_pp

In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from quick_pp.objects import Project
from quick_pp.rock_type import calc_r35_perm

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

all_data = project.get_all_data()
all_data['CPERM'] = all_data.CORE_PERM
all_data['CPORE'] = all_data.CORE_POR / 100

focused_well = 'HW-26'
well_data = all_data[all_data.WELL_NAME == focused_well].copy()

# Water Saturation Estimation

Water saturation estimation is crucial in petrophysics for several reasons:

1. **Hydrocarbon Volume Calculation**: It helps determine the volume of hydrocarbons in place. Accurate water saturation (Sw) values are essential for calculating the original oil in place (OOIP) and original gas in place (OGIP) volumes¹(https://petroshine.com/fluid-saturation/).
2. **Reservoir Characterization**: Understanding the distribution of water saturation helps in characterizing the reservoir, which is vital for planning production strategies and enhancing recovery¹(https://petroshine.com/fluid-saturation/).
3. **Production Forecasting**: Sw values are used in reservoir models to predict future production and to evaluate the economic viability of the reservoir²(https://www.mdpi.com/2077-1312/9/6/666).

### Methods to Estimate Water Saturation

1. **Resistivity Logs**: This is the most common method, where water saturation is estimated using resistivity measurements from well logs. The Archie equation is often used for clean sands, while modified versions like the Waxman-Smits model are used for shaly sands³(https://petrowiki.spe.org/Water_saturation_determination).
2. **Capillary Pressure Measurements**: Laboratory measurements of capillary pressure and corresponding water saturation provide detailed information about the pore structure and fluid distribution³(https://petrowiki.spe.org/Water_saturation_determination).
3. **Core Analysis**: Direct measurement of water saturation from core samples using techniques like the Dean-Stark method³(https://petrowiki.spe.org/Water_saturation_determination).
4. **Nuclear Magnetic Resonance (NMR)**: NMR logging tools can provide estimates of water saturation by measuring the response of hydrogen nuclei in the formation fluids³(https://petrowiki.spe.org/Water_saturation_determination).

This notebook estimates the water saturation using Archie equation and saturation height function based on the capillary pressure measurement.


***
# Log Derived Water Saturation

Estimation of Rw based on formation water salinity, assuming the depths are already in True Vertical Depth Sub Sea (TVDSS).
The range of Rw used in the original paper is 0.015 to 0.03 ohm.m

Estimation of cementation factor (m) based on pickett plot.

In [ ]:
from ipywidgets import widgets, interact

from quick_pp.saturation import pickett_plot

water_wells = ['HW-5', 'HW-7', 'HW-31']
focused_data = all_data[all_data.WELL_NAME.isin(water_wells)].copy()
# Use provided PHIE if estimated PHIT not available
focused_data['PHIT'] = focused_data['PHIT'].fillna(focused_data['PHIE'])

wells = widgets.SelectMultiple(
    options=['All'] + list(focused_data['WELL_NAME'].unique()),
    value=['All'],
    description='Wells:'
)
m = widgets.FloatSlider(
    value=2,
    min=1,
    max=3,
    step=.1,
    readout_format='.1f'
)
min_rw = widgets.FloatSlider(
    value=.01,
    min=.001,
    max=.1,
    step=.001,
    readout_format='.3f'
)
min_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.min(),
    min=focused_data.DEPTH.min(),
    max=focused_data.DEPTH.max() - 10,
    step=.1,
    readout_format='.1f'
)
max_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.max(),
    min=focused_data.DEPTH.min() + 10,
    max=focused_data.DEPTH.max(),
    step=.1,
    readout_format='.1f'
)

@interact(wells=wells, m=m, min_rw=min_rw, min_depth=min_depth, max_depth=max_depth)
def param(wells, m, min_rw, min_depth, max_depth):
    if 'All' in wells:
        data = focused_data[(focused_data.DEPTH >= min_depth) & (focused_data.DEPTH <= max_depth)]
    else:
        data = focused_data[(focused_data.WELL_NAME.isin(wells)) & (focused_data.DEPTH >= min_depth) & (focused_data.DEPTH <= max_depth)]
    pickett_plot(data['RT'], data['PHIT'], m=m, min_rw=min_rw)

In [ ]:
from quick_pp.saturation import estimate_temperature_gradient, estimate_rw_temperature_salinity, archie_saturation

# Debug water saturation
water_salinity = 2e5
m = 2.2

temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'imperial')
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)

swt = archie_saturation(well_data['RT'], rw, well_data['PHIT'], m=m)
swt = swt.clip(0, 1)

plt.figure(figsize=(15, 1.5))
plt.plot(well_data['DEPTH'], swt)
plt.ylim(0, 1.5)
plt.figure(figsize=(15, 1.5))
plt.plot(well_data['DEPTH'], rw, label='RW')
plt.yscale('log')
plt.legend()
plt.tight_layout()

***
# Saturation Height Function

## Core Data

Explaining the data source; measurement techniques

In [ ]:
from quick_pp.rock_type import calc_fzi, rock_typing, calc_r35, plot_fzi, plot_winland
from quick_pp.core_calibration import fit_j_curve, j_xplot, leverett_j, sw_shf_leverett_j, poroperm_xplot, pc_xplot

core_data = pd.read_csv(r'data\01_raw\COSTA\HW_core_data_all.csv')
core_data['CPORE'] = core_data['Phi (frac.)']
core_data['CPERM'] = core_data['K mD']
core_data['PC'] = core_data['O/B Pc (psia)']
core_data['PC_RES'] = core_data['O/B Pc (psia)'] * 0.088894  # oil-brine system
core_data['SW'] = core_data['Equiv Brine Sat. (Frac.)']
core_data['SWN'] = core_data.groupby('Sample')['SW'].transform(lambda x: (x - x.min()) / (1 - x.min()))

# Filter data
conditions = (
    (core_data['K mD'] > 0)
    & (core_data['Class'] == 'Good')
    # & (core_data['PC'] <= 40)
)
ori_core_data = core_data[conditions].copy()
core_data = core_data[conditions].copy()
# core_data.drop_duplicates(subset=['CPORE', 'CPERM', 'SW'], keep='last', inplace=True)

***
## Skelt Harrison Method using FZI Rock Types

Define the rock type based on Winland R35 cut-offs from previous notebook

#### QC capillary pressure data
Selecting a representative capillary pressure dataset for each rock type.

In [ ]:
# FZI
core_data = ori_core_data.copy()
fzi = calc_fzi(core_data['CPORE'], core_data['CPERM'])
fzi_cut_offs = [
    0.296, 0.469, 0.743, 1.05, 1.483, 1.866, 2.404, 3.724
]
rock_flag = rock_typing(fzi, fzi_cut_offs, higher_is_better=True)
core_data['ROCK_FLAG'] = rock_flag

plot_fzi(core_data['CPORE'], core_data['CPERM'], rock_type=rock_flag, cut_offs=fzi_cut_offs)
pd.DataFrame({
    'CPORE': core_data['CPORE'],
    'CPERM': core_data['CPERM'],
    'RRT': rock_flag,
    'fzi': fzi,
}).drop_duplicates(subset=['CPORE', 'CPERM'], keep='last').reset_index(drop=True).groupby(['RRT']).count()

In [ ]:
import matplotlib.pyplot as plt

# Get unique rock flags
unique_rock_flags = sorted(core_data['ROCK_FLAG'].unique())

# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))
axes = axes.flatten()

# Plot Pc vs SW for each rock flag
for i, rock_flag in enumerate(unique_rock_flags):
    ax = axes[i]
    data = core_data[core_data['ROCK_FLAG'] == rock_flag]
    for sample, sample_data in data.groupby('Sample'):
        ax.plot(sample_data['SW'], sample_data['PC_RES'], label=f'Sample {sample}')
    ax.set_ylabel('Pc (psia)')
    ax.set_xlabel('SW (frac)')
    ax.set_ylim(0, 40)
    ax.set_xlim(0, 1)
    ax.set_title(f'RRT {int(rock_flag)}')
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_calibration import pc_xplot, poroperm_xplot

rock_flag_widget = widgets.SelectMultiple(
    options=['All'] + sorted(list(core_data['ROCK_FLAG'].unique())),
    value=['All'],
    description='Rock Flag:'
)

@interact(rock_flag=rock_flag_widget)
def param(rock_flag):
    # Plot all data on poroperm plot
    poroperm_xplot(core_data['CPORE'], core_data['CPERM'])
    data = core_data[core_data.ROCK_FLAG.isin(rock_flag)] if any([l for l in rock_flag if l != 'All']) else core_data

    # Plot filtered data
    poroperm_data = data.drop_duplicates(subset=['CPORE', 'CPERM'], keep='last')
    poroperm_xplot(poroperm_data['CPORE'], poroperm_data['CPERM'], core_group=poroperm_data['Sample'])
    plt.show()
    for label, temp_df in data.groupby('Sample'):
        pc_xplot(temp_df['SW'], temp_df['PC_RES'], label=label, ylim=(0, 40))
    plt.show()
plt.close('all')

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_calibration import fit_skelt_harrison_curve, skelt_harrison_xplot

gw = 1.13
ghc = 0.87

sample = widgets.Dropdown(
    options=sorted(core_data['Sample'].unique()),
    description='Sample:'
)
conditions = (
    (core_data['PC'] < 1000)
    # & (core_data['SW'] < .99)
)
filtered_data = core_data[conditions].copy()
@interact(sample=sample)
def param(sample):
    data = filtered_data[filtered_data['Sample'] == sample]
    h = data['PC_RES'] / ((gw - ghc) * .433)
    a, b, c, d = fit_skelt_harrison_curve(data['SW'], h)
    skelt_harrison_xplot(
        data['SW'], data['PC_RES'], gw=gw, ghc=ghc, a=a, b=b, c=c, d=d,
        label=f'Sample {sample}:\n a:{a},\n b:{b},\n c:{c},\n d:{d}',
        ylim=(0, 100))

In [ ]:
skelt_params = {}
for sample, data in filtered_data.groupby('Sample'):
    h = data['PC_RES'] / ((gw - ghc) * .433)
    a, b, c, d = fit_skelt_harrison_curve(data['SW'], h)
    skelt_params[sample] = (a, b, c, d)

In [ ]:
# Convert j_params dictionary to DataFrame
j_params_df = pd.DataFrame.from_dict(
    skelt_params, orient='index', columns=['a', 'b', 'c', 'd']).reset_index().rename(columns={'index': 'Sample'})

# Merge filtered_data with j_params_df
merged_data = j_params_df.merge(
    filtered_data[['Sample', 'ROCK_FLAG']].drop_duplicates(), how='left', on='Sample')

# Group by ROCK_FLAG and sort by the first value of the parameters
sorted_samples = merged_data.groupby('ROCK_FLAG').apply(
    lambda x: x.sort_values(by='b')).reset_index(drop=True)

# Select samples considering values of 'b'
selected_samples = {}
used_a_values = set([0])
for rock_flag in sorted_samples['ROCK_FLAG'].unique():
    for _, row in sorted_samples[sorted_samples['ROCK_FLAG'] == rock_flag].sort_values(by='b').iterrows():
        if row['b'] not in used_a_values or all(row['b'] > val for val in used_a_values):
            selected_samples[rock_flag] = row
            used_a_values.add(row['b'])
            break
        else:
            selected_samples[rock_flag] = row

In [ ]:
import pprint

# Map rt_skelt_params with skelt_params key
mapped_fzi_skelt_params = pd.DataFrame(selected_samples).T.drop(columns=['ROCK_FLAG']).to_dict(orient='index')
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(mapped_fzi_skelt_params)

In [ ]:
from ipywidgets import interact, widgets

rt_widget = widgets.Dropdown(
    options=sorted(core_data['ROCK_FLAG'].unique()),
    description='Rock Type:'
)
ymax = widgets.FloatSlider(
    value=300,
    min=10,
    max=1000,
    step=10,
    description='Y Max:'
)

@interact(rt=rt_widget, ymax=ymax)
def param(rt, ymax):
    a = mapped_fzi_skelt_params[rt]['a']
    b = mapped_fzi_skelt_params[rt]['b']
    c = mapped_fzi_skelt_params[rt]['c']
    d = mapped_fzi_skelt_params[rt]['d']
    data = core_data[core_data['ROCK_FLAG'] == rt]
    skelt_harrison_xplot(
        data['SW'], data['PC_RES'], gw=gw, ghc=ghc,
        a=a, b=b, c=c, d=d, core_group=data['Sample'],
        label=f'{rt}: a:{a}, b:{b}, c:{c}, d:{d}', ylim=(0, ymax))

In [ ]:
# Get unique rock flags
unique_rock_flags = sorted(core_data['ROCK_FLAG'].unique())

# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))
axes = axes.flatten()

# Plot skelt_harrison_xplot for each rock flag
for i, rock_flag in enumerate(unique_rock_flags):
    ax = axes[i]
    data = core_data[core_data['ROCK_FLAG'] == rock_flag]
    a = mapped_fzi_skelt_params[rock_flag]['a']
    b = mapped_fzi_skelt_params[rock_flag]['b']
    c = mapped_fzi_skelt_params[rock_flag]['c']
    d = mapped_fzi_skelt_params[rock_flag]['d']
    for sample, sample_data in data.groupby('ROCK_FLAG'):
        ax = skelt_harrison_xplot(
            sample_data['SW'], sample_data['PC_RES'], gw=gw, ghc=ghc,
            a=a, b=b, c=c, d=d, core_group=sample_data['Sample'],
            label=f'a:{a}\nb:{b}\nc:{c}\nd:{d}', ylim=(0, 15), ax=ax)
    ax.set_title(f'RRT {int(rock_flag)}')
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

from quick_pp.core_calibration import skelt_harrison_func

# Plot mapped_fzi_params on the same j_xplot
for rock_flag, d in mapped_fzi_skelt_params.items():
    sample = d['Sample']
    a = d['a']
    b = d['b']
    c = d['c']
    d = d['d']
    
    h = np.geomspace(.001, 1000, 100)
    pci = h * (gw - ghc) * .433  # Convert g/cc to psi/ft
    plt.plot(skelt_harrison_func(h, a, b, c, d), pci,
            label=f'RRT{int(rock_flag)}, Sample {sample}\n a:{a}, b:{b}, c:{c}, d:{d}',
            linestyle='--')
plt.title('Skelt-Harrison J Function')
plt.xlabel('SW (v/v)')
plt.ylabel('Pc (psia)')
plt.xlim(0, 1)
plt.ylim(0, 10)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    

#### Estimate Free Water Level (FWL)

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from quick_pp.rock_type import calc_fzi_perm

well_data['LOG_RT'] = np.log10(well_data['RT'])
well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
well_data['GRN'] = MinMaxScaler().fit_transform(well_data[['GR']])

input_features = ['NPHI', 'NDI', 'LOG_RT', 'GRN']
# Predict PERM
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'rb') as file:
    fzi_model = pickle.load(file)
fzi_ml = 10**(fzi_model.predict(well_data[input_features]))
well_data['PERM'] = calc_fzi_perm(fzi_ml, well_data['PHIT'])

# Predict ROCK_FLAG
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_rt_model.qppm', 'rb') as file:
    fzi_rt_model = pickle.load(file)
well_data['ROCK_FLAG'] = fzi_rt_model.predict(well_data[input_features])

In [ ]:
from ipywidgets import interactive
from quick_pp.core_calibration import sw_skelt_harrison

fwl = 8600
fwl = widgets.FloatSlider(
    value=fwl,
    min=fwl / 1.1,
    max=fwl * 1.1,
    step=1
)

def plot(fwl):
    a = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['a'])
    b = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['b'])
    c = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['c'])
    d = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['d'])
    shf = sw_skelt_harrison(well_data.DEPTH, fwl, a, b, c, d)

    plt.figure(figsize=(20, 1))
    plt.plot(well_data['DEPTH'], swt, label='SWT')
    plt.plot(well_data['DEPTH'], shf, label='SHF')
    plt.ylim(0, 1.2)
    plt.legend()
    plt.figure(figsize=(20, 1))
    plt.plot(well_data['DEPTH'], well_data['ROCK_FLAG'], label='Rock Flag')
    plt.legend()

interactive_plot = interactive(plot, fwl=fwl)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

***
# Plot the results

In [ ]:
from quick_pp.plotter import plotly_log
fwl = 8618

a = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['a'])
b = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['b'])
c = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['c'])
d = well_data['ROCK_FLAG'].map(mapped_fzi_skelt_params).apply(lambda x: x['d'])
shf = sw_skelt_harrison(well_data.DEPTH, fwl, a, b, c, d)

# Plot individual results
well_data['SWT'] = swt
well_data['SHF'] = shf
fig = plotly_log(well_data, 'ft')
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()